In [1]:
#if alreasy installed, don't need to run following codes
#!conda update conda
#!conda config --add channels conda-forge
#!conda install spacy -y
#!python -m spacy download en

#!conda install scikit-learn -y
#!conda install beautifulsoup4 -y
#!conda install elasticsearch -y
#!pip install pyLDAvis

#for error ModuleNotFoundError: No module named 'cycler' when import matplotlib
#remember restart this notebook or using importlib.reload
#!conda remove matplotlib -y
#!conda remove cycler -y
#!pip uninstall cycler -y
#!pip uninstall matplotlib -y
#!pip install cycler
#!pip install matplotlib
!pip install memory_profiler

In [8]:
from sklearn.model_selection import train_test_split
import time
import multiprocessing
import warnings
warnings.filterwarnings(action='once')
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams["figure.figsize"] = [16,9]
  
from Models import DataLoadModel
from Utils import utilTools,utilDataPreprocess,utilLDA
import importlib
importlib.reload(DataLoadModel)
importlib.reload(utilTools)
importlib.reload(utilLDA)
importlib.reload(utilDataPreprocess)

IDFILE="./idlist_cleaned.pkl"
DATAFILE_CLEANED="./preprocessed_data_cleaned.pkl"
DATAFILE_TOKENIZED="./preprocessed_data_tokenized.pkl"
DATAFILE_VECTORIZED="./preprocessed_data_vectorized.pkl"
VECTORIZERFILE = "./vectorizer.pkl"

In [2]:
def saveCleanedData(idlist, data):
    if idlist is None or data is None:
         raise ValueError('you must input two valid objects')
    try:
        utilTools.pickleDump(IDFILE, idlist)
        utilTools.pickleDump(DATAFILE_CLEANED, data)
    except Exception as e:
        print(e)
        print("save failed, idfile={}, dataFile={}".format(IDFILE, DATAFILE_CLEANED))
        raise e
        
# EOFError: Ran out of input, if file size is 0
def loadCleanedData():
    try:
        idlist = utilTools.pickleLoad(IDFILE)
        data = utilTools.pickleLoad(DATAFILE_CLEANED)
    except Exception as e:
        print(e)
        print("load failed, idfile={}, dataFile={}".format(IDFILE, DATAFILE_CLEANED))
        raise e
        
    return idlist, data  

In [3]:
def saveVectorizedData(data, vectorizer):
    if idlist is None or data is None or vectorizer is None:
         raise ValueError('you must input two valid objects')
    try:
        utilTools.pickleDump(DATAFILE_VECTORIZED, data)
        utilTools.dillDump(VECTORIZERFILE, vectorizer)
    except Exception as e:
        print(e)
        print("save failed,dataFile={}, vectorizerFile={}".format(DATAFILE_VECTORIZED, VECTORIZERFILE))
        raise e
        
def loadVectorizedData():
    try:
        data = utilTools.pickleLoad(DATAFILE_VECTORIZED)
        vectorizer = utilTools.dillLoad(VECTORIZERFILE)
    except Exception as e:
        print(e)
        print("load failed, dataFile={}, vectorizerFile={}".format(DATAFILE_VECTORIZED, VECTORIZERFILE))
        raise e
        
    return data, vectorizer

In [4]:
def loadAndCleanData(force_reload=False, n_jobs=1, force_reload_file=None, force_reload_count=-1):
    try:
        if force_reload:
            raise ValueError("will force reload data...")
        start_time = time.time()
        idlist, data = loadCleanedData()
        end_time = time.time()
        print("load {} preprocessed cleaned records in {} Seconds".format(len(idlist), end_time - start_time))
    except Exception as e:
        print("load preprocessed data failed, Will retrieve from dataset...")
        start_time = time.time()
        data = DataLoadModel.loadStackoverflowFromXML(force_reload_file, force_reload_count)
        #data = DataLoadModel.iterLoadStackoverflowFromES(1000)
        end_time = time.time()
        print("Retrieved {} records in {} Seconds".format(len(data), end_time - start_time))
        
        start_time = time.time()
        X_train = [row[1] for row in data]
        idlist = [row[0] for row in data]
        cleaner = utilDataPreprocess.CleanTextTransformer(n_jobs=n_jobs)
        data = cleaner.fit_transform(X_train)
        try:
            saveCleanedData(idlist, data)
            pass
        except Exception as e:
            print(e)
            print("pickleDump cleaned data Failed")
        end_time = time.time()
        print("clean data in {} Seconds".format(end_time - start_time))
        
    return idlist, data

In [5]:
#https://stackoverflow.com/questions/11025005/python-sharing-a-dictionary-between-parallel-processes
#https://stackoverflow.com/questions/5442910/python-multiprocessing-pool-map-for-multiple-arguments
def tokenizeAndVectorieData(data_in, force_reprocess=False, n_jobs=1, min_df=1, max_df = 1.0, max_features=20000):
    #tokenized
    try:
        if force_reprocess:
            raise ValueError("will force reprocess, tokenizing data...")
        start_time = time.time()
        tokenized_data= utilTools.pickleLoad(DATAFILE_TOKENIZED)
        end_time = time.time()
        print("load preprocessed tokenized data in {} Seconds".format(end_time - start_time))
    except Exception as e:
        print("load preprocessed tokenized data failed, Will reprocess...")
        start_time = time.time()
        pool = multiprocessing.Pool(n_jobs)
        tokenized_data = pool.map(utilDataPreprocess.tokenizeText, data_in)
        pool.close()
        pool.join()
        try:
            utilTools.pickleDump(DATAFILE_TOKENIZED, tokenized_data)
        except Exception as e:
            print(e)
            print("pickleDump tokenized data Failed")
        end_time = time.time()
        print("tokenized data in {} Seconds".format(end_time - start_time))
        
    #vectorize, everytime should revectorize for the tokenized data
    vectorized_data, vectorizer = utilDataPreprocess.countvectorizeDataWithTokens(tokenized_data, 
                                                                                  min_df=min_df, 
                                                                                  max_df=max_df,
                                                                                  max_features=max_features)
    try:
        saveVectorizedData(vectorized_data, vectorizer)
    except Exception as e:
        print("pickleDump vectorized data Failed")
        
    return tokenized_data, vectorized_data, vectorizer

In [6]:
def vectorizData(data_in, force_reprocess=False, n_jobs=1, min_df=1, max_df = 1.0, max_features=20000):
    tokens=None
    try:
        if force_reprocess:
            raise ValueError("will force reprocess data...")
        start_time = time.time()
        data, vectorizer = loadVectorizedData()
        end_time = time.time()
        print("load preprocessed vectorized data in {} Seconds".format(end_time - start_time))
    except Exception as e:
        print(e)
        print("load preprocessed vectorized data failed, Will reprocess...")
        tokens, data, vectorizer= tokenizeAndVectorieData(data_in, njobs=n_jobs, min_df=min_df, max_df=max_df, max_features=max_features)
        
    return tokens, data, vectorizer

In [ ]:
%time
%load_ext memory_profiler
#%mprun
%memit

if __name__ == "__main__":
    #Parse XML [F:\stackoverflow.com-Posts\Posts.xml]Done, total [1001] records!
    #Retrieved 1001 records in 0.1178131103515625 Seconds
    #clean data in 2.963881731033325 Seconds
    #idlist, cleaned_data = loadAndCleanData(force_reload=True, n_jobs=4, 
    #                                        force_reload_file=r"F:\stackoverflow.com-Posts\Posts.xml",
    #                                        force_reload_count=10000)
    idlist, cleaned_data = loadAndCleanData() 
    #cleaned_data_dict=dict(zip(idlist, cleaned_data))
    print("\n")
    
    #tokenized data in 13.708552837371826 Seconds
    #vectorize done in 0.02807474136352539 Seconds
    #tokenized_data, vecterized_data, vectorizer = tokenizeAndVectorieData(cleaned_data,
    #                                                                      force_reprocess = True,
    #                                                                      n_jobs=4,
    #                                                                      min_df=5, #5
    #                                                                      max_df=0.6, #0.6
    #                                                                      max_features=20000)
    tokenized_data, vecterized_data, vectorizer = vectorizData(cleaned_data, n_jobs=4,min_df=1,max_df=1.0,max_features=20000)
    #if tokenized_data:
    #    tokenized_data_dict=dict(zip(idlist, tokenized_data))
    #vecterized_data_dict=dict(zip(idlist, vecterized_data))
    print("\n")
    
    #n_jobs=-1, Trained LDA in 303.9054253101349 Seconds
    #n_jobs=1, Trained LDA in 146.29173946380615 Seconds
    start_time = time.time()
    lda = utilLDA.trainLDA(vecterized_data, n_topics=10, n_jobs=1)
    try:
        utilTools.dillDump("./lda.pkl", lda)
    except:
        pass        
    feature_names = vectorizer.get_feature_names()
    utilLDA.print_top_words(lda, feature_names, 20)
    end_time = time.time()
    print("Trained LDA in {} Seconds".format(end_time - start_time))
    
    #get document_topic_distribution
    doc_topic_distr = lda.transform(vecterized_data[:15])
    utilLDA.visDocTopicDist(doc_topic_distr)
    print(doc_topic_distr.shape)
    print(idlist[0])
    print(doc_topic_distr[0])
    
    #utilLDA.visLDA(lda, vecterized_data, vectorizer, utilTools.getIP(), 8889)
    utilLDA.visLDA(lda, vecterized_data, vectorizer, "127.0.0.1",8889)

Wall time: 0 ns
The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler
peak memory: 271.36 MiB, increment: 0.00 MiB
load 10001 preprocessed cleaned records in 0.011043310165405273 Seconds


load preprocessed vectorized data in 0.016041994094848633 Seconds


